# Algothon 2024: Exploratory Data Analysis
---------

## Understanding the data

In [5]:
import pandas as pd

# Load the data using pandas
data_path = 'prices.txt'

# creates data frame from text file
# a dataframe (df) is like a table, that is an object is python 
# that we can get info from
prices_df = pd.read_csv(data_path, delim_whitespace=True, header=None)

# get dimensions (aka row and column sizes) of our dataframe
df_shape = prices_df.shape

# Display the first and last few rows of the 
# DataFrame to understand its structure
df_head = prices_df.head()

df_shape, df_head


((500, 50),
       0      1      2      3      4      5      6      7      8      9   ...  \
 0  13.46  71.65  48.46  50.52  52.10  13.00  18.98  47.71  69.49  49.96  ...   
 1  13.48  72.10  48.52  50.50  52.06  12.95  18.95  47.84  69.73  49.93  ...   
 2  13.47  72.35  48.48  50.62  51.80  12.79  18.98  47.98  69.60  49.33  ...   
 3  13.53  72.51  48.42  50.75  51.66  12.66  18.96  48.74  69.54  49.67  ...   
 4  13.64  71.99  48.40  50.65  51.97  12.62  18.89  48.88  69.68  49.46  ...   
 
       40     41     42     43     44     45     46     47     48     49  
 0  32.64  55.76  14.46  58.94  36.71  52.62  49.33  36.22  49.00  56.09  
 1  32.52  55.97  14.44  59.81  36.64  52.58  49.20  36.27  48.84  56.08  
 2  32.48  56.34  14.50  59.04  36.89  52.49  49.48  36.39  48.56  55.90  
 3  32.59  56.32  14.40  58.73  36.94  52.40  49.42  36.41  49.00  56.14  
 4  32.64  56.32  14.36  59.01  37.03  52.44  49.79  36.42  48.14  55.90  
 
 [5 rows x 50 columns])

We can see there are 500 rows and 50 columns. 

Each column represents an instrument (stock). There are 50 instruments, so there are 50 columns. 
Each row represents the price of each instrument (stock) on a particular day. We have been given 500 days of data, so there are 500 rows. 

So prices.txt is training data. We want to build a model that reflects this data without overfitting to it. Then we will be given more data that our model can hopefully also work on. 

##### Convert to numPy array for the purpose of the algorithm

In [6]:
# Convert the DataFrame to a NumPy array
prices_array = prices_df.values
prices_array

array([[13.46, 71.65, 48.46, ..., 36.22, 49.  , 56.09],
       [13.48, 72.1 , 48.52, ..., 36.27, 48.84, 56.08],
       [13.47, 72.35, 48.48, ..., 36.39, 48.56, 55.9 ],
       ...,
       [13.69, 67.83, 46.66, ..., 33.6 , 33.75, 54.22],
       [13.55, 67.61, 46.73, ..., 33.79, 33.53, 54.5 ],
       [13.4 , 67.33, 46.72, ..., 33.82, 33.4 , 55.01]])